In [1]:
import os
import time
import pathlib
import yaml
from datetime import datetime

In [2]:
now = datetime.now()
dt_string = now.strftime("%d%B%Y")
dt_string

'13December2021'

In [3]:
startTime = time.time()
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locations_27112021_12:05:17.yaml')
group_runs = 500
ngroups = 15  
#------------------------------------------------------------
# Global variables
#------------------------------------------------------------
oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']

In [3]:
#------------------------------------------------------------
# Output directory
#------------------------------------------------------------
# add date tag to sbatch .list filename
now = datetime.now()
dt_string = now.strftime("%d%B%Y")
output_list_dir=f'/scratch/rmueller/MIDOSS/Results/{dt_string}' 

# create output directory if it doesn't already exist
try:
    os.makedir(output_list_dir)
except:
    print(f'{output_list_dir} already exists')
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_oil_spills_{dt_string}.list')
#------------------------------------------------------------
# Load yaml file name with list of output netcdf files to aggregate
#------------------------------------------------------------
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for oil in oil_types:
    n_iter[oil]=max(1,len(run_paths[oil])/group_runs)
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
with open(sbatch_file, 'w') as f:
    for oil in oil_types: # loop through oils
        this_iter = 1
        print(f'*** {oil} (Number of iterations: {n_iter[oil]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[oil]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            #------------------------------------------------------------
            # write to file the sbatch submission command
            f.write(f"python aggregate_oil_spills.py '{yaml_file}' {oil} {first} {last} '{output_list_dir}'\n")
            # #------------------------------------------------------------
            # # Define output filename
            # aggregated_csv = output_list_dir / f'{oil}_runset{this_iter}_filenames.csv'
            # #------------------------------------------------------------
            # # Save output netcdf and a .csv file containing list of runs
            # beaching.to_netcdf(aggregated_beaching_nc, engine='h5netcdf')
            # surface.to_netcdf(aggregated_surface_nc, engine='h5netcdf')
            # #numpy.savetxt(aggregated_csv, files, delimiter =", ", fmt ='% s')
            this_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {oil}: {executionTime/60:.2f}')
f.close

/scratch/rmueller/MIDOSS/Results/10December2021 already exists
*** akns (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for akns: 0.07
*** bunker (Number of iterations: 6.696) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Execution time in minutes for bunker: 0.07
*** dilbit (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for dilbit: 0.07
*** jet (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for jet: 0.07
*** diesel (Number of iterations: 12.478) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Execution time in minutes for diesel: 0.07
*** gas (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for gas: 0.07
*** other (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for other: 0.07


<function TextIOWrapper.close()>

### Create a secondary file for runs that didn't complete 

In [4]:
output_list_dir=f'/scratch/rmueller/MIDOSS/Results/10December2021' 
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_oil_spills_LONG2_{dt_string}.list')
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for oil in oil_types:
    n_iter[oil]=max(1,len(run_paths[oil])/group_runs)


In [5]:
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
with open(sbatch_file, 'w') as f:
    for oil in oil_types: # loop through oils
        this_iter = 1
        print(f'*** {oil} (Number of iterations: {n_iter[oil]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[oil]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            netcdf_file = pathlib.Path(
                    output_list_dir,
                    f'beaching_{oil}_{first}-{last}.nc')
            if os.path.exists(netcdf_file)==False:
                f.write(f"python aggregate_oil_spills.py '{yaml_file}' {oil} {first} {last} '{output_list_dir}'\n")
            this_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {oil}: {executionTime/60:.2f}')
f.close

*** akns (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for akns: 0.40
*** bunker (Number of iterations: 6.696) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Execution time in minutes for bunker: 0.41
*** dilbit (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for dilbit: 0.41
*** jet (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for jet: 0.41
*** diesel (Number of iterations: 12.478) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Execution time in minutes for diesel: 0.44
*** gas (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for gas: 0.44
*** other (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for other: 0.44


<function TextIOWrapper.close()>